In [ ]:
# imports

import sys
import os
import numpy as np
import pandas as pd
from skimage.transform import warp_polar
import matplotlib.pyplot as plt
import importlib

sys.path.append('../')
import plotting.plottools
importlib.reload(plotting.plottools)
from plotting.plottools import plot_histogram
import rebinning
importlib.reload(rebinning)
from rebinning import rebin_keep_zero
import averaging
importlib.reload(averaging)
from averaging import average_outer_ring
from averaging import average_rings

In [ ]:
# get some example histograms

fname = '../data/examples/ZeroBias-Run2023C-PromptReco-v1-DQMIO-PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1.parquet'
df = pd.read_parquet(fname)
xbins = df['Xbins'][0]
ybins = df['Ybins'][0]
nhists = 5
hists = np.array([df['histo'][i].reshape(xbins,ybins) for i in range(nhists)])
runs = np.array(df['fromrun'])
lumis = np.array(df['fromlumi'])
entries = np.array(df['entries'])
print('Shape of hists array: {}'.format(hists.shape))
print('Runs: {}'.format(runs))
print('Lumis: {}'.format(lumis))
print('Entries: {}'.format(entries))

In [ ]:
# plot raw data

for i, hist in enumerate(hists): 
    fig, ax = plot_histogram(hist)
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)

In [ ]:
# make average of outer rings and plot

hists = average_rings(hists, radii={0:2, 50:5})
for i, hist in enumerate(hists):
    fig, ax = plot_histogram(hist)
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}, averaged'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)

In [ ]:
# do warping and plot

warped_hists = []
for i in range(len(hists)):
    warped_hists.append(warp_polar(hists[i], center=(99,99), order=0))
warped_hists = np.array(warped_hists)
warped_hists = warped_hists[:,:,22:82]
print(hists.shape)
print(warped_hists.shape)

for i, hist in enumerate(warped_hists):
    fig, axs = plt.subplots(ncols=2, width_ratios=[1,3], figsize=(18,6))
    plot_histogram(hists[0], fig=fig, ax=axs[0])
    plot_histogram(hist, fig=fig, ax=axs[1], docolorbar=False)
    axs[0].text(0.02, 1.02, 'Run: {}, lumi: {}'.format(runs[i], lumis[i]), transform=axs[0].transAxes, fontsize=12)

In [ ]:
# do rebinning on one example histogram

hist = warped_hists[1]
rebinned_hist = rebin_keep_zero(hist, (40,30))
plot_histogram(rebinned_hist)

In [ ]:
# do rebinning for all histograms at once

rebinned_hists = rebin_keep_zero(warped_hists, (40, 30))
for i, hist in enumerate(rebinned_hists):
    fig, ax = plot_histogram(hist)
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}, rebinned'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)